In [2]:
from shapely.geometry import Point, LineString, Polygon
import folium
import pandas as pd
import geopandas as gpd
from gsheets import Sheets
%matplotlib inline
pd.set_option('display.max_columns', 500)

In [3]:
#loads pandas dataframe from google sheets
whole_nst_sheet = 'https://docs.google.com/spreadsheets/d/1_QY-l4xhMu5nZVluprOgRs6rUzgkkBemapdsg5lFzKU'
sheets = Sheets.from_files('client_secrets.json')
s = sheets.get(whole_nst_sheet)
main_sheet = s.sheets[1]
df = main_sheet.to_frame(index_col=False)

In [7]:
df.head()

,Title,Date,"Community (city,town, ward)",LGA,State,Total Deaths,Boko Haram (P),State Actor (P),Sectarian Actor (excluding BH) (P),Other Armed Actor (P),Kidnapper (P),Robber (P),Other (P),Election-related Actor (P),Cameroon State Actor (P),Boko Haram (V),State Actor (V),Sectarian Actor (V),Other Armed Actor (V),Political Actor (V),Kidnapper (V),Kidnapee (V),Robber (V),Journalist (V),Civilian (V),Election-related Actor (V),Cameroon State Actor,Bomb,Gun,Machete,Suicide Bombing,Other Weapon,TK,Drinking Establishment,Goverment Building,Church,Mosque,Bank,School,Other Location,Notes,Sources 1,Sources 2,Sources 3,Latitude,Longitude,full place name
0,Army Barracks Attack Ahead of Goodluck Jonatha...,5/29/2011,Bauchi,Bauchi,Bauchi,20.0,Boko Haram,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,Bomb,NaN,NaN,NaN,NaN,NaN,Drinking Establishment,Government Building,NaN,NaN,NaN,NaN,NaN,NaN,http://allafrica.com/stories/201105310776.html,http://www.reuters.com/article/2011/05/30/us-n...,http://www.vanguardngr.com/2011/06/boko-haram-...,10.6371093,10.0807298,"Bauchi, Bauchi, Nigeria"
1,Kaduna,5/29/2011,Federal Capital Territory,Zaria,Kaduna,6.0,Boko Haram,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,Bomb,NaN,NaN,NaN,NaN,NaN,Drinking Establishment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://weeklytrust.com.ng/?option=com_content&...,http://allafrica.com/stories/201105311096.html,http://allafrica.com/stories/201105310901.html,11.0855409,7.719945399999999,"Zaria, Kaduna, Nigeria"
2,Morning Attack in Zaria,5/30/2011,"Iyan Juma, Zaria City",Zaria,Kaduna,2.0,Boko Haram,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,Bomb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other,NaN,http://weeklytrust.com.ng/?option=com_content&...,http://weeklytrust.com.ng/?option=com_content&...,http://weeklytrust.com.ng/?option=com_content&...,11.0855409,7.719945399999999,"Zaria, Kaduna, Nigeria"
3,Shehu of Borno's Brother Killed,5/30/2011,"Gangamar, Shehuri North Ward",Maiduguri,Borno,1.0,Boko Haram,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gun,NaN,NaN,NaN,Targeted Killing,NaN,NaN,NaN,NaN,NaN,NaN,Other,NaN,http://www.google.com/hostednews/afp/article/A...,http://allafrica.com/stories/201106010765.html,NaN,11.8310981,13.1509672,"Maiduguri, Borno, Nigeria"
4,Attempted Attack on Government Vaccine Warehouse,6/2/2011,Unguwan Doki,Maiduguri,Borno,0.0,Boko Haram,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Bomb,NaN,NaN,NaN,NaN,NaN,NaN,Government Building,NaN,NaN,NaN,NaN,NaN,NaN,http://dailytimes.com.ng/article/boko-haram-se...,http://allafrica.com/stories/201106030574.html,http://www.google.com/hostednews/afp/article/A...,NaN,NaN,"Maiduguri, Borno, Nigeria"


In [4]:
#sets data as index and converts to datetime object
df = df.set_index('Date')
df.index = pd.to_datetime(df.index)

In [5]:
#filters data for a week long amount of time
cw = df.loc['2019-02-09': '2019-02-15']

In [6]:
#converts lat long in DF to shapely point geometry
cw['geo'] = cw.apply(lambda x: Point(float(x['Latitude']), float(x['Longitude'])), axis=1)

/Users/aschharwood/anaconda3/envs/NST/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [33]:
cw.head()

,Title,"Community (city,town, ward)",LGA,State,Total Deaths,Boko Haram (P),State Actor (P),Sectarian Actor (excluding BH) (P),Other Armed Actor (P),Kidnapper (P),Robber (P),Other (P),Election-related Actor (P),Cameroon State Actor (P),Boko Haram (V),State Actor (V),Sectarian Actor (V),Other Armed Actor (V),Political Actor (V),Kidnapper (V),Kidnapee (V),Robber (V),Journalist (V),Civilian (V),Election-related Actor (V),Cameroon State Actor,Bomb,Gun,Machete,Suicide Bombing,Other Weapon,TK,Drinking Establishment,Goverment Building,Church,Mosque,Bank,School,Other Location,Notes,Sources 1,Sources 2,Sources 3,Latitude,Longitude,full place name,geo
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-02-09,9 Vehicles Set Ablaze as Hoodlums Attack FCT A...,Dei-Dei,NaN,Federal Capital Territory,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Election-related Actor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other,NaN,https://www.thisdaylive.com/index.php/2019/02/...,https://guardian.ng/politics/hoodlums-attack-f...,https://www.vanguardngr.com/2019/02/abuja-9-ca...,8.9062309,7.189050198,", Federal Capital Territory, Nigeria",POINT (8.906230900000001 7.189050198)
2019-02-09,Army Repels Insurgents’ Attack on Adamawa Town,NaN,Madagali,Adamawa,5.0,Boko Haram,State Actor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,Gun,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other,NaN,https://www.thisdaylive.com/index.php/2019/02/...,https://www.vanguardngr.com/2019/02/troops-foi...,https://punchng.com/soldier-village-head-kille...,10.81351,13.45938969,"Madagali, Adamawa, Nigeria",POINT (10.81351 13.45938969)
2019-02-09,Soldiers kill 2 at Imo APC rally,Ejemekwuru,Oguta,Imo,2.0,NaN,State Actor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,Gun,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other,NaN,https://www.vanguardngr.com/2019/02/soldiers-k...,http://thenationonlineng.net/two-dead-as-rival...,https://nigeriatoday.ng/crime/soldiers-alleged...,5.7112598,6.805809975,"Oguta, Imo, Nigeria",POINT (5.7112598 6.805809975)
2019-02-10,"Armed bandits invade Niger communities, kill n...","Banda, Malfa and Tunga",Shiroro,Niger,9.0,NaN,NaN,NaN,Other Armed Actor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other,NaN,https://www.sunnewsonline.com/armed-bandits-in...,http://thenationonlineng.net/armed-bandits-kid...,https://www.dailytrust.com.ng/senator-umaru-de...,9.8744602,6.709259987,"Shiroro, Niger, Nigeria",POINT (9.8744602 6.709259987)
2019-02-11,Suspected cultist stoned to death in Benue,NaN,Makurdi,Benue,1.0,NaN,NaN,NaN,Other Armed Actor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other,NaN,https://www.sunnewsonline.com/suspected-cultis...,https://guardian.ng/news/mob-stones-suspected-...,NaN,7.7264299,8.526080132,"Makurdi, Benue, Nigeria",POINT (7.7264299 8.526080132000001)


In [7]:
#creates a dict with place name as key and point geometry as value
place_doct = dict(zip(cw['full place name'], cw['geo']))

In [36]:
place_doct

{', Federal Capital Territory, Nigeria': <shapely.geometry.point.Point at 0x12433c8d0>,
 'Madagali, Adamawa, Nigeria': <shapely.geometry.point.Point at 0x12433c3c8>,
 'Oguta, Imo, Nigeria': <shapely.geometry.point.Point at 0x12433c6d8>,
 'Shiroro, Niger, Nigeria': <shapely.geometry.point.Point at 0x11b6990f0>,
 'Makurdi, Benue, Nigeria': <shapely.geometry.point.Point at 0x12433c668>,
 'Iwo, Osun, Nigeria': <shapely.geometry.point.Point at 0x12433c748>,
 'Ekiti East, Ekiti, Nigeria': <shapely.geometry.point.Point at 0x12433c7f0>,
 'Owerri Municipal, Imo, Nigeria': <shapely.geometry.point.Point at 0x12433c358>,
 'Aba South, Abia, Nigeria': <shapely.geometry.point.Point at 0x12433c5f8>,
 'Abeokuta South, Ogun, Nigeria': <shapely.geometry.point.Point at 0x12433c2e8>,
 'Ngala, Borno, Nigeria': <shapely.geometry.point.Point at 0x12433c1d0>,
 'Ovia South-West, Edo, Nigeria': <shapely.geometry.point.Point at 0x12433c518>,
 'Ndokwa West, Delta, Nigeria': <shapely.geometry.point.Point at 0x12433

In [8]:
#agregates total number of deaths by place
LGA_TD = pd.DataFrame(cw.groupby('full place name')['Total Deaths'].sum()).reset_index()

In [9]:
#maps the geometries back onto those place names

LGA_TD['geometry'] = LGA_TD.apply(lambda x: place_doct[x['full place name']], axis=1)

In [10]:
#converts DF to a geopandas DF

gdf = gpd.GeoDataFrame(LGA_TD, geometry='geometry', crs={'init': 'epsg:4326'})

In [11]:
#converts geopandas df to a Geojson file
points_gjson = folium.features.GeoJson(gdf, name = "Total Deaths")

In [12]:
#creates a folum base map
m = folium.Map(location=[7.17, 8.83],
    zoom_start=5, control_scale=True)

In [13]:
#adds points to basemap
points_gjson.add_to(m)

In [14]:
m

In [51]:
m.save('nst_weekly.html')